In [1]:

from ibmdbpy import IdaDataBase, IdaDataFrame

# @hidden_cell
# This connection object is used to access your data and contains your credentials.
# You might want to remove those credentials before you share your notebook.
idadb_b17dd4d9dff84b519a9e627f3bec50c4 = IdaDataBase(dsn='DASHDB;Database=BLUDB;Hostname=dashdb-txn-sbox-yp-dal09-08.services.dal.bluemix.net;Port=50000;PROTOCOL=TCPIP;UID=sdc18363;PWD=g6tf3xjbqm+p2hxd')

sql_data = IdaDataFrame(idadb_b17dd4d9dff84b519a9e627f3bec50c4, 'SDC18363.TRANSLATIONS').as_dataframe()
sql_data.head()

# You can close the database connection with the following code. Please keep the comment line with the @hidden_cell tag,
# because the close function displays parts of the credentials.
# @hidden_cell
# idadb_b17dd4d9dff84b519a9e627f3bec50c4.close()
# To learn more about the ibmdby package, please read the documentation: http://pythonhosted.org/ibmdbpy/


,Customer Type,Comments,Identified_Language,Translated_Language,Response,Categorisation,Sentiment,Sector,Flight No,Flight Date,Cabin Class,Seat No,Aircraft Version,Tail No,FFP Member,Gender,Age,Country
0,Passive,Amenity Kit: Comments,fr,Very well.,Très bien.,Compliment,Positive,MRU-CDG,NK0034,26/11/2018,ECONOMY,039E,359C,3BNBQ,NO,Male,26-34,France
1,Detractor,Amenity Kit: Comments,en,Not taking well kids,Not taking well kids,No adequate products,Negative,MRU-MAA,NK0746,26/11/2018,ECONOMY,None,343E,3BNBJ,NO,Male,35-44,Mauritius
2,Detractor,Amenity Kit: Comments,fr,Up to the detail we see the regression of Air ...,jusque dans le détail nous voyons la régressio...,Poor quality/ Very dissatisfied,Negative,BOM-MRU,NK0749,26/11/2018,ECONOMY,011C,343E,3BNBI,NO,Female,45-54,Mauritius
3,Detractor,Amenity Kit: Comments,en,I appreciate this kit in economy. It is a welc...,I appreciate this kit in economy. It is a welc...,Compliment,Positive,SIN-MRU,NK0647,27/11/2018,ECONOMY,016B,332,3BNBM,NO,Female,55-64,Mauritius
4,Detractor,Amenity Kit: Comments,en,Nothing,Nothing,None,Neutral,MRU-BOM,NK0748,27/11/2018,ECONOMY,026H,343C,3BNAU,YES,Male,45-54,India


In [2]:
# Checking Shape of dataframe(Number of records & Coulmns Available in Dataframe)
sql_data.shape

(5347, 18)

In [3]:
#upgrade Watson Packages
! pip install --upgrade "ibm-watson>=4.3.0"

     |████████████████████████████████| 368kB 8.9MB/s eta 0:00:01
     |████████████████████████████████| 204kB 29.2MB/s eta 0:00:01
  Stored in directory: /home/dsxuser/.cache/pip/wheels/73/21/91/61a59270c329be2edc947c69871c25c50bf7dd87ebbfe07cdc
  Stored in directory: /home/dsxuser/.cache/pip/wheels/6a/42/50/f96888116b329578304f9dda4693cef6f3e76e18272d22cb6c
Successfully built ibm-watson ibm-cloud-sdk-core


In [4]:
# Import libraries for Sentiments extraction using Watson Language Understanding Service From IBM Cloud
#import pandas as pd
#import json
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator
from ibm_watson.natural_language_understanding_v1 import Features, EntitiesOptions, KeywordsOptions, ConceptsOptions, EmotionOptions, SentimentOptions
from ibm_watson import ApiException
import operator

In [5]:
# Adding Watson Natural Language Understanding credentials 
authenticator = IAMAuthenticator('A1GtyTmtA04fXb6pULb4RkEjZ9bVPJsMUie4YZA8j1jf')
natural_language_understanding = NaturalLanguageUnderstandingV1(version='2019-07-12',authenticator=authenticator
)

natural_language_understanding.set_service_url('https://api.eu-gb.natural-language-understanding.watson.cloud.ibm.com/instances/71485e6d-eb98-4ade-bf42-904a7765003a')

In [6]:
# Dummy List's For Sentiment Extraction
sentiment=[]
sentiment_score=[]
keywords=[]
keywords_label=[]
keywords_relevance=[]


emotion_sadness=[]
emotion_joy=[]
emotion_fear=[]
emotion_disgust=[]
emotion_anger=[]


concepts_text=[]
concepts_relevance=[]

In [7]:
from tqdm import tqdm

In [ ]:
for i in tqdm(range(0,len(sql_data['Translated_Language']))):
    try:
        response = natural_language_understanding.analyze(text=str(sql_data['Translated_Language'][i]),
        language='en',                          
        features=Features(
        entities=EntitiesOptions(emotion=True, sentiment=True, limit=1),
        keywords=KeywordsOptions(emotion=True, sentiment=True,limit=1),
        concepts=ConceptsOptions(limit=1),
        emotion=EmotionOptions(document=True),
        sentiment=SentimentOptions(document=True))).get_result()

     #sentiments list collection    
        try:
            (response['sentiment']['document']['label']) != ''  
            sentiment.append(response['sentiment']['document']['label'])   
        except:
            sentiment.append('')
        try:   
            response['sentiment']['document']['score'] != ''
            sentiment_score.append(response['sentiment']['document']['score'])    
        except:       
            sentiment_score.append('')

    #keywords list collections
        try:
            (response['keywords'][0]['text']) != ''       
            keywords.append((response['keywords'][0]['text']))
        except:        
            keywords.append('')
        try:        
            (response['keywords'][0]['sentiment']['label']) != ''     
            keywords_label.append(response['keywords'][0]['sentiment']['label'])
        except:        
            keywords_label.append('')

        try:           
            (response['keywords'][0]['relevance']) != ''         
            keywords_relevance.append(response['keywords'][0]['relevance'])
        except:        
            keywords_relevance.append('')

    #emotion extraction

        try:
            (response['emotion']) != []      
            emotion_sadness.append(response['emotion']['document']['emotion']['sadness'])
            emotion_anger.append(response['emotion']['document']['emotion']['anger'])
            emotion_joy.append(response['emotion']['document']['emotion']['joy'])
            emotion_disgust.append(response['emotion']['document']['emotion']['disgust'])
            emotion_fear.append(response['emotion']['document']['emotion']['fear'])

        except:
            emotion_anger.append('')
            emotion_disgust.append('')
            emotion_fear.append('')
            emotion_joy.append('')
            emotion_sadness.append('')

    #extracting concepts
        try:
            response['concepts'] != []
            concepts_text.append(response['concepts'][0]['text'])
#             print(concepts_text)
            concepts_relevance.append(response['concepts'][0]['relevance'])
#             print(concepts_relevance)
                
        except:
            
            concepts_relevance.append('')
            concepts_text.append('')
    except:
#         print(False + i)
        sentiment.append('')
        sentiment_score.append('')
        keywords.append('')
        keywords_label.append('')
        keywords_relevance.append('')


        emotion_sadness.append('')
        emotion_joy.append('')
        emotion_fear.append('')
        emotion_disgust.append('')
        emotion_anger.append('')


        concepts_text.append('')
        concepts_relevance.append('')

 21%|██        | 1108/5347 [07:46<24:51,  2.84it/s] ERROR:root:invalid request: content is empty
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: invalid request: content is empty, Code: 400 , X-global-transaction-id: 10553ae2636675db648c053c32986121
 33%|███▎      | 1790/5347 [12:34<25:59,  2.28it/s]ERROR:root:Error in service call
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py", line 377, in _make_request
    httplib_response = conn.getresponse(buffering=True)
TypeError: getresponse() got an unexpected keyword argument 'buffering'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages

 71%|███████▏  | 3815/5347 [27:09<05:00,  5.10it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 71%|███████▏  | 3816/5347 [27:09<04:49,  5.29it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 71%|███████▏  | 3817/5347 [27:10<07:30,  3.40it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo

 72%|███████▏  | 3838/5347 [27:16<07:02,  3.57it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 72%|███████▏  | 3839/5347 [27:17<08:10,  3.08it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 72%|███████▏  | 3840/5347 [27:17<07:42,  3.26it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo

 72%|███████▏  | 3861/5347 [27:24<08:43,  2.84it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 72%|███████▏  | 3862/5347 [27:24<07:40,  3.23it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 72%|███████▏  | 3863/5347 [27:24<07:00,  3.53it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo

 73%|███████▎  | 3884/5347 [27:31<06:44,  3.62it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 73%|███████▎  | 3885/5347 [27:31<06:32,  3.72it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 73%|███████▎  | 3886/5347 [27:32<06:28,  3.76it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo

...

 74%|███████▍  | 3953/5347 [27:47<04:54,  4.74it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 74%|███████▍  | 3954/5347 [27:48<04:58,  4.66it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 74%|███████▍  | 3955/5347 [27:48<05:03,  4.59it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo

 74%|███████▍  | 3976/5347 [27:52<04:51,  4.70it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 74%|███████▍  | 3977/5347 [27:52<04:56,  4.62it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 74%|███████▍  | 3978/5347 [27:53<04:57,  4.60it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo

 75%|███████▍  | 3999/5347 [27:57<04:58,  4.51it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 75%|███████▍  | 4000/5347 [27:57<04:58,  4.52it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 75%|███████▍  | 4001/5347 [27:58<04:57,  4.52it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo

 75%|███████▌  | 4022/5347 [28:06<09:43,  2.27it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 75%|███████▌  | 4023/5347 [28:06<08:26,  2.62it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 75%|███████▌  | 4024/5347 [28:06<07:22,  2.99it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo

 76%|███████▌  | 4045/5347 [28:13<06:36,  3.29it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 76%|███████▌  | 4046/5347 [28:13<05:49,  3.73it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_response=response)
ibm_cloud_sdk_core.api_exception.ApiException: Error: Forbidden, Code: 403
 76%|███████▌  | 4047/5347 [28:13<06:50,  3.16it/s]ERROR:root:Forbidden
Traceback (most recent call last):
  File "/opt/conda/envs/Python36/lib/python3.6/site-packages/ibm_cloud_sdk_core/base_service.py", line 229, in send
    response.status_code, error_message, http_respo